In [2]:
import pandas as pd

In [3]:
import sys
sys.path.append('../')

In [4]:
from metaexplainercode import metaexplainer_utils
from metaexplainercode import codeconstants

In [5]:
user_responses = codeconstants.SYNTHESIS_FOLDER + '/eval_files/user_responses/'

In [6]:
all_subfolders = metaexplainer_utils.get_subfolders_in_folder(user_responses)

In [8]:
system_responses_list = []
per_explanation_response_list = []
user_responses_list = []

for response_folder in all_subfolders:
    system_responses_list.append(pd.read_excel(response_folder + '/OverallSystemEvaluationSet.xlsx'))

In [17]:
#which dimensions are from trust, curiosity and satisfication

system_responses_df = pd.concat(system_responses_list)

print(system_responses_df.groupby(['Evaluation Question']).mean())

                                                    Likert Rating
Evaluation Question                                              
I am confident in the MetaExplainer. I feel tha...       1.777778
I like using the system for decision making. [S...       1.666667
I want to know more about what the AI did.               4.111111
I want to know what the AI would have done if s...       4.111111
I want to know why the AI did not make some oth...       4.000000
I want to understand what the AI will do next.           3.555556
I was surprised by the AI’s actions and want to...       3.000000
The MetaExplainer can perform the task better t...       2.444444
The outputs of the MetaExplainer are predictabl...       3.222222
